In [5]:
!pipenv install opencv-python

Installing opencv-python...
Resolving opencv-python...
Added opencv-python to Pipfile's [packages] ...
✔ Installation Succeeded-python...
⠋ Installing opencv-python...
Pipfile.lock (d841b9) out of date, updating to (ff91d1)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠸ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (dc9ac101c7117b44308bb3134f17a8f7bf6b2e361c8e0375780405869aff91d1)!
Installing dependencies from Pipfile.lock (ff91d1)...


In [32]:
!pipenv install pyautogui

Installing pyautogui...
Resolving pyautogui...
Added pyautogui to Pipfile's [packages] ...
✔ Installation Succeededgui...
⠋ Installing pyautogui...
Pipfile.lock (ff91d1) out of date, updating to (1e9306)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠸ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (b75283a316123f11d94bca0c6492cf86a2179f22904491fdf85d2556111e9306)!
Installing dependencies from Pipfile.lock (1e9306)...


In [60]:
!pipenv install Pillow

Installing Pillow...
Resolving Pillow...
Added pillow to Pipfile's [packages] ...
✔ Installation Succeeded...
⠋ Installing Pillow...
Pipfile.lock (1e9306) out of date, updating to (4412a3)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠸ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (af38ab80e426fc97db7b24af945beed2d71bd09e4f028586f6ada825e04412a3)!
Installing dependencies from Pipfile.lock (4412a3)...


In [61]:
!pipenv install mss

Installing mss...
Resolving mss...
Added mss to Pipfile's [packages] ...
✔ Installation Succeeded
⠋ Installing mss...
Pipfile.lock (4412a3) out of date, updating to (2e4ab4)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠋ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (921fc1b354d0aecdaeb71ea370b136fa4d6f385e0abd5c821512ab86022e4ab4)!
Installing dependencies from Pipfile.lock (2e4ab4)...


In [40]:
import cv2
import numpy as np
import pyautogui

In [41]:
water_img = cv2.imread('water.png', cv2.IMREAD_UNCHANGED)
lure_img = cv2.imread('lure.png', cv2.IMREAD_UNCHANGED)
catch_img = cv2.imread('catch.png', cv2.IMREAD_UNCHANGED)

In [51]:
cv2.imshow('Water', water_img)
cv2.waitKey()
cv2.destroyAllWindows()

In [52]:
# result = cv2.matchTemplate(water_img, catch_img, cv2.TM_CCOEFF_NORMED)
result = cv2.matchTemplate(water_img, lure_img, cv2.TM_SQDIFF_NORMED)

In [53]:
# cv2.imshow('Result', result)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [54]:
minv, maxv, minl, maxl = cv2.minMaxLoc(result)

In [55]:
# maxl

In [56]:
# w = lure_img.shape[1]
# h = lure_img.shape[0]

In [57]:
# cv2.rectangle(water_img, maxl, (maxl[0] + w, maxl[1] + h), (0,255,255), 2)

In [58]:
# pyautogui.position()

In [59]:
pyautogui.moveTo(x=maxl[0], y=maxl[1], duration=0.7)